In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import re
from numpy import NaN


header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
    }

dta = pd.read_csv('letterboxd\watched.csv')[['Name','Letterboxd URI']]

pttrn1 = r'[0-9][.][0-9][0-9]'
pttrn2 = r'>[a-z]+<'
pttrn3 = r'[0-9]+'
pttrn4 = r'>[A-Za-z,()]+<'

comp1 = re.compile(pattern=pttrn1)
comp2 = re.compile(pattern=pttrn2)
comp3 = re.compile(pattern=pttrn3)
comp4 = re.compile(pattern=pttrn4)

data = dict()

for dt in tqdm(dta.values):
    r = requests.get(dt[1], headers=header)
    try:
      site = BeautifulSoup(r.text,'html.parser')
      meta_string = str(site.find_all('meta',{'name':'twitter:data2'})[0])
      avr_rating = comp1.findall(meta_string)[0]
    except IndexError:
       avr_rating=NaN

    try:
      raw_duration = [str(x) for x in site.find_all('p', {'class':'text-link text-footer'})][0]
      duration = comp3.findall(raw_duration)[0]
    except:
      duration = NaN
    try:
      raw_genre = site.find_all('div', {'class':'text-sluglist capitalize'})[0]
      raw_str = ''.join([str(x) for x in raw_genre.find_all('a')])
      genre = [x[1:-1] for x in comp2.findall(raw_str)] 
    except:
       genre=[]
    
    try:
      language = str(site.select('a[href^="/films/language"]')[0])
      lang = comp4.findall(language)[0][1:-1]

    except:
      lang=NaN


    data[f'{dt[0]}'] = [avr_rating,duration,genre,lang]
    sleep(3)

100%|██████████| 398/398 [30:29<00:00,  4.60s/it]


In [73]:
year = pd.read_csv(r'letterboxd\watched.csv')
rating = pd.read_csv(r'letterboxd\ratings.csv')

In [74]:
rating

,Date,Name,Year,Letterboxd URI,Rating
0,2021-08-02,Harakiri,1962,https://boxd.it/1NTU,5.0
1,2021-08-02,You Can't Take It with You,1938,https://boxd.it/1ejM,3.0
2,2021-08-02,Rocky,1976,https://boxd.it/28Uk,3.5
3,2021-08-02,Easy A,2010,https://boxd.it/17DK,2.0
4,2021-08-02,Avengers: Age of Ultron,2015,https://boxd.it/3qi6,1.5
...,...,...,...,...,...
271,2023-06-06,Bicycle Thieves,1948,https://boxd.it/23hW,4.5
272,2023-06-19,It's Such a Beautiful Day,2012,https://boxd.it/hCY2,2.5
273,2023-06-22,Spider-Man: Across the Spider-Verse,2023,https://boxd.it/kSz4,3.0
274,2023-06-25,Sunset Boulevard,1950,https://boxd.it/2a20,3.5


In [71]:
raw = pd.DataFrame(data).T
raw = raw.reset_index()
raw.columns = ['Name','Avr_Rating','Duration','Genre','Language']

In [110]:
clean = pd.merge(raw,rating, on='Name', how='outer')
clean

,Name,Avr_Rating,Duration,Genre,Language,Date,Year,Letterboxd URI,Rating
0,About Time,3.95,123,"[fantasy, romance, drama]",English,2023-02-14,2013.0,https://boxd.it/43bI,3.5
1,Harakiri,4.67,135,"[history, drama, action]",Japanese,2021-08-02,1962.0,https://boxd.it/1NTU,5.0
2,Joker,3.82,122,"[crime, drama, thriller]",English,2022-08-22,2019.0,https://boxd.it/h4cS,3.5
3,Joan of Arc,3.24,145,"[drama, war, history]",English,2021-11-09,1948.0,https://boxd.it/UsY,3.5
4,The Trial of Joan of Arc,3.75,62,"[history, drama]",French,2022-05-29,1962.0,https://boxd.it/HGe,3.5
...,...,...,...,...,...,...,...,...,...
393,Evangelion: 1.0 You Are (Not) Alone,3.64,101,"[action, drama, animation]",Japanese,NaN,NaN,NaN,NaN
394,The Apartment,4.42,125,"[comedy, romance, drama]",English,2023-06-06,1960.0,https://boxd.it/2aGE,5.0
395,It's Such a Beautiful Day,4.38,62,"[animation, comedy, drama]",English,2023-06-19,2012.0,https://boxd.it/hCY2,2.5
396,Spider-Man: Across the Spider-Verse,4.55,140,"[animation, adventure, action]",English,2023-06-22,2023.0,https://boxd.it/kSz4,3.0
